# CUDA-Q

## What is the Computational Configuration of the Supercomputing?

In [1]:
!nvidia-smi #topo -m 

Thu Mar 27 14:48:28 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:30:00.0 Off |                    0 |
| N/A   32C    P0    24W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  On   | 00000000:6D:00.0 Off |                    0 |
| N/A   

## How to start the Anaconda Environment CUDA-Q on Kuatomu?

### `using python`

```cpp
Steps:

1) ssh kuatomu
2) module load anaconda3/4.13.0
3) source activate cudaq-env
4) unset PYTHONPATH
```

### `using C++`

```cpp
Steps:

1) ssh kuatomu
2) module load anaconda3/4.13.0
3) source activate cudaq-env
4) unset PYTHONPATH
5) source /opt/nvidia/cudaq/set_env.sh
```

## Sample test CUDA-Q on Kuatomu

### `python`

In [2]:
%%writefile sample-cudaq.py
import cudaq

qubit_count = 2

print(cudaq.draw(kernel, qubit_count))

results = cudaq.sample(kernel, qubit_count)
# Should see a roughly 50/50 distribution between the |00> and
# |11> states. Example: {00: 505  11: 495}

print("Measurement distribution:" + str(results))

Writing sample-cudaq.py


In [3]:
!python sample-cudaq.py

NameError: name 'kernel' is not defined


## `C++`

In [1]:
%%writefile sample-cudaq.cpp
#include <cudaq.h>

struct bell {
  int operator()(int num_iters) __qpu__ {
    cudaq::qarray<2> q;
    int nCorrect = 0;
    for (int i = 0; i < num_iters; i++) {
      h(q[0]);
      x<cudaq::ctrl>(q[0], q[1]);
      auto results = mz(q);
      if (results[0] == results[1])
        nCorrect++;

      reset(q[0]);
      reset(q[1]);
    }
    return nCorrect;
  }
};

int main(int argc, char **argv) { 
    
  printf("N Correct = %d\n", bell{}(100)); 
 
  return 0;
   
}

Writing sample-cudaq.cpp


In [2]:
!nvq++ sample-cudaq.cpp -o sample-cudaq

In [3]:
!./sample-cudaq

N Correct = 100
